In [2]:
import json
import geopandas as gp
from shapely.geometry import Point, LineString, Polygon

In [3]:
# these are the unedited data from Ollie O'Brien's vis repo (https://github.com/oobrien/vis)
with open("static/tfl_lines.geojson", 'r') as f:            
    gj = json.loads(f.read()) 

In [4]:
# Some segments belong to multiple lines
# We need to build new LineStrings with each of these
# this results in a small amount of duplication but that's OK
new_features = []
for feature in gj['features']:
    num_lines = len(feature['properties']['lines'])
    if num_lines > 1:
        for segment in feature['properties']['lines'][1:]:
            new_feature = feature
            new_feature['properties']['line_name'] = segment['name']
            new_feature['properties']['shared'] = True
            new_features.append(
                new_feature
            )
    else:
        feature['properties']['shared'] = False

In [5]:
# assign a new line_name property to existing features
for feature in gj['features']:                              
    feature['properties']['line_name'] = feature['properties']['lines'][0]['name']

In [6]:
# combine existing and new features
gj['features'] = gj['features'] + new_features

In [7]:
# remove 'lines' property, since we don't need it any more
for feature in gj['features']:
    try:
        del feature['properties']['lines']
    except KeyError:
        pass

In [8]:
with open("static/tfl_lines_edited.geojson", 'w') as g:
    json.dump(gj, g)

In [9]:
df = gp.read_file("static/tfl_lines_edited.geojson")

In [10]:
df.head()

,id,shared,line_name,geometry
0,LeaValleyLine1,False,London Overground,LINESTRING (-0.059409291146068 51.523912138223...
1,LeaValleyLine2,False,London Overground,LINESTRING (-0.057597366770253 51.532454060856...
2,LeaValleyLine3,False,London Overground,LINESTRING (-0.05785044994051 51.5411764951798...
3,LeaValleyLine4,False,London Overground,LINESTRING (-0.061160397519048 51.549089532848...
4,LeaValleyLine5,False,London Overground,LINESTRING (-0.06845384240135299 51.5586559609...


In [11]:
# Mapbox GL can only extrude polygons.
# LineStrings can be converted into Polygons by buffering them
# choose an appropriate width here
df.geometry = df.geometry.buffer(0.000100)

In [12]:
# fix up other unfamiliar lines
df['line_name'] = df['line_name'].replace('East London', 'London Overground')
df['line_name'] = df['line_name'].replace('Crossrail 2', 'Crossrail')
df['line_name'] = df['line_name'].replace('TfL Rail', 'Crossrail')

In [13]:
df.line_name.unique()

array(['London Overground', 'Victoria', 'Piccadilly', 'District',
       'Circle', 'Hammersmith & City', 'Metropolitan', 'Central',
       'Jubilee', 'DLR', 'Bakerloo', 'Northern', 'Waterloo & City',
       'Crossrail', 'Tramlink', 'Emirates Air Line',
       'Thameslink 6tph line'], dtype=object)

In [14]:
df.to_file("static/tube_polygons.geojson", driver="GeoJSON")